## Dependency 문제 해결법
해당 방법은 제 맥북 맥os 기준이므로, 윈도우나 다른 환경에서는 해결 방법이 조금 다를 수 있습니다.
1. torchtext.data는 0.10.0 이후로 outdated 됐으므로 그 이하 버전을 설치해야합니다. 저는 검색해서 문서가 나오는 0.8.0을 선택했습니다.
2. torchtext는 최신 파이토치와 호환이 안되므로 버전에 맞춰서 설치해야합니다. 
torchtext와 호환되는 파이토치 버전은 1.7, 파이썬 버전은 3.6<=python<=3.8이므로 이에 맞춰 환경을 설치했습니다.
참고 링크: [version compatibility](https://pypi.org/project/torchtext/)
3. 그런데 또 파이썬 3.8에선 무슨 이유에선지 파이토치 1.7 버전이 깔리지 않아 파이썬을 3.7 버전으로 내리고 설치에 성공했습니다.
판단 근거: 분명 [파이썬 3.9 이상에서 설치가 안된다](https://stackoverflow.com/questions/75823700/i-am-unable-to-install-pytorch-1-7-1-using-pip-conda)고 하는데 3.8에서도 안되는 것으로 추론하여 버전을 낮췄습니다.
4. 파이썬 3.7은 환경 생성을 바로 할 수 없으므로 [다음 링크](https://velog.io/@hotsun1508/Error-%ED%95%B4%EA%B2%B0-M2%EC%97%90-%EC%95%84%EB%82%98%EC%BD%98%EB%8B%A4-python-3.7-%EA%B0%80%EC%83%81%ED%99%98%EA%B2%BD-%EC%83%9D%EC%84%B1)를 참고했습니다.
5. 마지막으로 conda-forge 채널을 통해 spacy를 설치했습니다. en_core_web_sm 모델도 같이 받아야합니다.
그런데 다음 오류를 겪었습니다:  
OSError: [E941] Can't find model 'en'. It looks like you're trying to load a model from a shortcut, which is obsolete as of spaCy v3.0. To load the model, use its full name instead:
spacy 3.x 이상의 버전 문제라고 판단하여, 버전도 2.1.0으로 pip를 통해 낮춰서 설치했습니다.
이후 동일하게 en 모델을 찾을 수 없다는 오류가 떴습니다. 그래서 언어 모델도 en_core_web_sm이 아니라, 다음 명령어로 'en' 모델을 설치하여 최종적으로 문제를 해결했습니다.  
```python -m spacy download en```

## Embeddings in Practice

### Preprocessing

In [1]:
import torch.nn.functional as F
import torch.nn as nn
from torch import optim

from torchtext import *
import torchtext

In [2]:
import torch
# 비맥북의 경우
# dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# 맥북의 경우 pytorch 1.7.0에서 gpu 연산을 지원하지 않습니다.
dev = torch.device("cpu")

In [3]:
# set up fields
TEXT = data.Field(lower=True, include_lengths=True, \
batch_first=False, tokenize='spacy')
LABEL = data.LabelField()

# make splits for data
train, test = datasets.IMDB.splits(TEXT, LABEL)

# build the vocabulary
TEXT.build_vocab(train, vectors='glove.6B.100d') \
# use 'glove.42B.300d' for greater accuracy or 'glove.6B.100d' for greater speed
LABEL.build_vocab(train)

# make iterator for splits
train_iter, test_iter = data.BucketIterator.splits((train, test), \
batch_sizes=(128,1024), device=dev, sort_within_batch=True, repeat=False)

/opt/anaconda3/envs/nlpbook/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/opt/anaconda3/envs/nlpbook/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/opt/anaconda3/envs/nlpbook/lib/python3.7/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.

### Model

In [4]:
class RNN_classifier(torch.nn.Module):
    def __init__(self, embedding_size = 100, hidden_size = 512, num_layers = 3):
        super().__init__()

        # Set up an embedding layer with the right dimensions, and copy the weights from the pretrained glove embeddings
        vocab = TEXT.vocab
        self.embed = torch.nn.Embedding(len(vocab), embedding_size)
        self.embed.weight.data.copy_(vocab.vectors)

        # Set up a standard PyTorch RNN sections with the right dimensions and a variable number of layers
        self.rnn = torch.nn.RNN(embedding_size, hidden_size, num_layers)

        # Add a two layer classification head with the right dimensions. The final layer must output a single number
        self.classificationLayer1 = torch.nn.Linear(hidden_size,10)
        self.classificationLayer2 = torch.nn.Linear(10,1)


    def forward(self, input, lengths=None):

        embed_input = self.embed(input)
        packed_emb = torch.nn.utils.rnn.pack_padded_sequence(embed_input, \
        lengths, batch_first=False)

        output, hidden = self.rnn(packed_emb)
        hidden = hidden[-1]
        x = hidden.squeeze(0)
        x = self.classificationLayer1(x)
        x = self.classificationLayer2(x)

        logits = x.view(-1)
        return logits


In [5]:
model = RNN_classifier(hidden_size=256, num_layers=1)
model.to(dev)

RNN_classifier(
  (embed): Embedding(101980, 100)
  (rnn): RNN(100, 256)
  (classificationLayer1): Linear(in_features=256, out_features=10, bias=True)
  (classificationLayer2): Linear(in_features=10, out_features=1, bias=True)
)

In [6]:
for batch in train_iter:
    (x,x_len) = batch.text
    pred = model(x,x_len)
    print(pred.shape)
    break

/opt/anaconda3/envs/nlpbook/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


torch.Size([128])


### Training

In [7]:
loss_func = F.binary_cross_entropy_with_logits
opt = optim.Adam(model.parameters(), lr=1e-4)
epochs = 1

In [8]:
def get_metrics(model, test_data):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(test_data):
            text, text_lengths = batch_data.text
            logits = model(text, text_lengths)
            predicted_labels = (torch.sigmoid(logits) > 0.5).long()
            total += batch_data.label.size(0)
            correct += (predicted_labels == batch_data.label.long()).sum()
        return correct.float()/total

In [9]:
from tqdm.notebook import tqdm

# 주피터 노트북의 경우 ipywidgets 필요(conda, pip)

for epoch in tqdm(range(epochs)):
    model.train()
    for batch in tqdm(train_iter):
        (x,x_lengths)=batch.text
        pred = model(x,x_lengths)

        actual=batch.label.float()
        loss = loss_func(pred,actual)

        loss.backward()
        opt.step()
        opt.zero_grad()

    if (epoch==5):
        for g in opt.param_groups:
            g['lr'] = 3e-3

    print("Accuracy: " + str(get_metrics(model, test_iter).cpu().numpy()))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

Accuracy: 0.66336


### Validation

In [10]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    # based on:
    # https://github.com/bentrevett/pytorch-sentiment-analysis/blob/
    # master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]

    tensor = torch.LongTensor(indexed).to(dev)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [11]:
review = """I like that Far From Home is trying something new and that its
humor  feels more real than the ironic cracks in most superhero movies.
I just wish its good pieces all came together more satisfyingly."""

print('Probability positive:')
predict_sentiment(model, review)

Probability positive:


0.5359222888946533